In [ ]:
#!pip install googletrans==4.0.0-rc1
#!pip install langid

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import re
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt 
from transformers import BertTokenizer,AutoTokenizer
from transformers import BertModel,AutoModel
import torch
from torch import nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from tqdm import tqdm
from googletrans import Translator
%matplotlib inline

import seaborn as sns
import langid
from tqdm import tqdm
tqdm.pandas()

# Data Description
For this competition, we curated a dataset comprised of around 4.5K book summaries labeled in seven genre categories: </br>Fiction (0), </br>Thriller (1), </br>Childrens-Book (2), </br>Political (3), </br>Science-Fiction (4), </br>War (5), and </br>Motivational (6).

In [ ]:
#importing the data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
#train data
print(train)

In [ ]:
#test data
test.head()

In [ ]:
#looking for the missing data
train.isna().sum()

In [ ]:
#before removing missing values 
train.shape, test.shape

In [ ]:
#remove missing vlaues
train.dropna(inplace=True)
test.dropna(inplace=True)
train.shape, test.shape

In [ ]:
#visualize the distribution of data
plt.hist(train['Label'],bins=20)
plt.show()

In [ ]:
plt.hist(train['genre'],bins=20)
plt.xticks(rotation=45)
plt.show()

In [ ]:
#print a random train sample
index = np.random.randint(0,len(train))
print(f"index : {index}")
train['summary'][index]

In [ ]:
# ***************** Main Data Cleaning function***************************
def clean_data(text):
    text=text.replace('\n',"")
    text=text.replace('\r',"")
    text=re.sub(r'https?://\S+|www\.\S+',' ',text)
    cleaned_text = re.sub('[^a-zA-Z0-9\u0980-\u09FF,./!?$%&"\'\\-_;।]'," ",text) #remove unusual symbols such as emoji : # % @ * + etc 
    cleaned_text = re.sub('(?<=[\s][^\u0980-\u09FFa-zA-Z0-9])[^\u0980-\u09FFA-Za-z0-9]+',"", cleaned_text) #remove multiples punctuations after space(doesnt remove the space)
    cleaned_text = re.sub('(?<=[^\u0980-\u09FFa-zA-Z\s0-9])[^\u0980-\u09FFA-Za-z0-9]'," ", cleaned_text) #remove multiple punctuations just keeps the first one(no space before the punctuation)
    cleaned_text = re.sub('\s(?=[।,.?!;])',"", cleaned_text) #remove the space before the punctuatuion mark
    cleaned_text = ' '.join(cleaned_text.split()) #removes multiple spaces keeps only one 
    cleaned_text = re.sub(r'([।,.?!;])(?=[\u0980-\u09E5a-zA-Z0-9\u09F0-\u09FF])',r'\1 ', cleaned_text) #Make data standardized like there will e a space after a punctuation
    cleaned_text = re.sub('(?<=[\u0980-\u09E5\u09F0-\u09FF])[.][\s](?=[\u0980-\u09E5\u09F0-\u09FF])',' ', cleaned_text) #removes . from bangla text except from bangla digits
   
    return cleaned_text

In [ ]:
#test the clean function
index = np.random.randint(0,len(train))
# index = 2933
print(f"Index : {index}")
clean_data(train['summary'][index])

# Create new features using cleaned data

In [ ]:
tqdm.pandas()
train['text']=train.summary.progress_apply(clean_data)
train

In [ ]:
#calculating sentence length 
train['length'] = [len(x.split()) for x in train['text']]
train

In [ ]:
# Dropping rows that got less datas
train.drop(train[train['length'] < 50].index, inplace=True)

In [ ]:
train.reset_index(inplace=True)
train

In [ ]:
train[train['Label']==2].summary.iloc[21]

# Translate the english too bangla

In [ ]:
from googletrans import Translator

def translate_to_bengali(text):
    translator = Translator()
    translation = translator.translate(text, src='en', dest='bn')
    return translation.text

In [ ]:
holder=[]
def ultimate_translator(text):
    flag = re.search('[a-zA-Z0-9]+',text)
    if flag is not None: #is there is english word in the text
        flag1 = re.search('[\u0980-\u09FF]+',text)
        if flag1 is None: # only english so direct translate
            temp = translate_to_bengali(text)
            holder.append(temp)
            return temp
        else: #mix of bangla and english
            converted = ""
            for words in text.split():
                flag3 = re.search('[a-zA-Z0-9]',str(words))
                if flag3 is not None: # the word is english so translate
                    converted += translate_to_bengali(words)
                    converted +=' '
                else: # the word is bangla direct add
                    converted += words
                    converted +=' '
            holder.append(converted)
            return converted
    else: #no english text  
        holder.append(text)
        return text    

In [ ]:
#check the translator function
ultimate_translator(train['text'][0])

In [ ]:
tqdm.pandas()
train['text'].progress_apply(ultimate_translator)

In [ ]:
def final_clean(text):
    text=re.sub("'","",text)
    pattern = r'[a-zA-Z0-9]'
    
    if re.match(pattern, text):
        if re.search(r'[ক-হ]', text):
            text = re.sub(pattern, " ", text)
        else:
            text = translate_to_bengali(text)
            text = re.sub(pattern, " ", text)
    
    return text

In [ ]:
tqdm.pandas()
train['summary2']=train['text'].progress_apply(final_clean)

In [ ]:
#calculating sentence length 
train['length'] = [len(x.split()) for x in train['text']]
train

In [ ]:
# calculating rows with smaller length of sentence:
smaller_train = len(train[train['length']<1])
smaller_train

In [ ]:
train[train['length']<1]

In [ ]:
# Dropping rows that got less datas
train.drop(train[train['length'] < 1].index, inplace=True)

In [ ]:
train

In [ ]:
#delete all only english cell for now 
# Define a regex pattern for Bengali characters (Unicode range)
bangla_pattern = re.compile(r'[\u0980-\u09FF]+')

# Filter rows where 'text' column contains at least one Bengali character
train = train[train['text'].apply(lambda x: bool(bangla_pattern.search(x)))]
train

In [ ]:
#now do same for the test

In [ ]:
# same for test
tqdm.pandas()
test['text']=test.summary.progress_apply(clean_data)
test

In [ ]:
test['summary2']=test['text'].progress_apply(final_clean)

In [ ]:
test['length'] = [len(x.split()) for x in test['text']]
test

In [ ]:
# calculating rows with smaller length of sentence:
smaller_test = len(test[test['length']<1])
smaller_test

In [ ]:
test[test['length']<6]

In [ ]:
#test how to remove only english datas

# Sample DataFrame (replace this with your actual DataFrame)
data = {'text': ['আমি বাংলা ভাষা ভালবাসি', 'I love Bengali আসদফ anguage', 'Some, English text', 'কিছু বাংলা লেখা']}
df = pd.DataFrame(data)

# Define a regex pattern for Bengali characters (Unicode range)
bangla_pattern = re.compile(r'[\u0980-\u09FF]+')

# Filter rows where 'text' column contains at least one Bengali character
df = df[df['text'].apply(lambda x: bool(bangla_pattern.search(x)))]

print(df)